# Implementation of search algorithms

## Graph class and map utility functions

In [45]:
# Sample code from https://www.redblobgames.com/pathfinding/a-star/
# Copyright 2014 Red Blob Games <redblobgames@gmail.com>
#
# Feel free to use this code in your own projects, including commercial projects
# License: Apache v2.0 <http://www.apache.org/licenses/LICENSE-2.0.html>

from __future__ import annotations
# some of these types are deprecated: https://www.python.org/dev/peps/pep-0585/
from typing import Protocol, Iterator, Tuple, TypeVar, Optional
T = TypeVar('T')

Location = TypeVar('Location')
class Graph(Protocol):
    def neighbors(self, id: Location) -> list[Location]: pass   #a graph contains a function neighbor that, given a location, identifies a list of neighboring locations

class SimpleGraph:
    def __init__(self):
        self.edges: dict[Location, list[Location]] = {}     #a simplegraph is initialized by specifiying only the edges as a dictionary of vertex: neighbouring vertices (outgoing edges)

    def neighbors(self, id: Location) -> list[Location]:
        return self.edges[id]                               #the neighbors method is then easily obtained by calling edge on that node

import collections

# utility functions for dealing with square grids
def from_id_width(id, width):
    return (id % width, id // width)

def draw_tile(graph, id, style):
    r = " ⋅ "
    if id in graph.swamps: r = "⋅⋅⋅"
    if 'number' in style and id in style['number']: r = " %-2d" % style['number'][id]
    if 'point_to' in style and style['point_to'].get(id, None) is not None:
        (x1, y1) = id
        (x2, y2) = style['point_to'][id]
        if x2 == x1 + 1: r = " > "
        if x2 == x1 - 1: r = " < "
        if y2 == y1 + 1: r = " ∨ "
        if y2 == y1 - 1: r = " ∧ "
    if 'path' in style and id in style['path']:   r = " @ "
    if 'start' in style and id == style['start']: r = " S "
    if 'goal' in style and id == style['goal']:   r = " G "
    if id in graph.walls: r = "###"
    return r

def draw_grid(graph, **style):
    print("___" * graph.width)
    for y in range(graph.height):
        for x in range(graph.width):
            print("%s" % draw_tile(graph, (x, y), style), end="")
        print()
    print("---" * graph.width)

# data from main article
DIAGRAM1_WALLS = [from_id_width(id, width=30) for id in [21,22,51,52,81,82,93,94,111,112,123,124,133,134,141,142,153,154,163,164,171,172,173,174,175,183,184,193,194,201,202,203,204,205,213,214,223,224,243,244,253,254,273,274,283,284,303,304,313,314,333,334,343,344,373,374,403,404,433,434]]
GridLocation = Tuple[int, int]

class SquareGrid:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.walls: list[GridLocation] = []
        self.swamps: list[GridLocation] = []

    def in_bounds(self, id: GridLocation) -> bool:
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height

    def passable(self, id: GridLocation) -> bool:
        return id not in self.walls

    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)] # E W N S
        # see "Ugly paths" section for an explanation:
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return results

def reconstruct_path(came_from: dict[Location, Location],
                     start: Location, goal: Location) -> list[Location]:

    current: Location = goal
    path: list[Location] = []
    if goal not in came_from: # no path was found
        return []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start) # optional
    path.reverse() # optional
    return path

g = SquareGrid(30, 15)
g.walls = DIAGRAM1_WALLS

start = (8, 7)
goal = (18, 8)

draw_grid(g, start=start, goal=goal)

__________________________________________________________________________________________
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ############### ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ############### ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  S  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  G  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ ###### ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 

In [46]:
from os import get_blocking
class WeightedGraph(Graph):
    def cost(self, from_id: Location, to_id: Location) -> float: pass   #this is going to be called in the algos

class GridWithWeights(SquareGrid):
    def __init__(self, width: int, height: int):
        super().__init__(width, height)
        self.weights: dict[GridLocation, float] = {}

    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        if to_node in self.swamps:
            return self.weights.get(to_node, 5)
        return self.weights.get(to_node, 1)


D4_walls = [(1, 7), (1, 8), (2, 7), (2, 8), (3, 7), (3, 8)]
D4_swamp = {loc: 5 for loc in [(3, 4), (3, 5), (4, 1), (4, 2),
                                       (4, 3), (4, 4), (4, 5), (4, 6),
                                       (4, 7), (4, 8), (5, 1), (5, 2),
                                       (5, 3), (5, 4), (5, 5), (5, 6),
                                       (5, 7), (5, 8), (6, 2), (6, 3),
                                       (6, 4), (6, 5), (6, 6), (6, 7),
                                       (7, 3), (7, 4), (7, 5)]}


start, goal = (1, 4), (8, 3)
g = GridWithWeights(11, 10)
g.walls = D4_walls
g.swamps = D4_swamp
draw_grid(g, start=start, goal=goal)

_________________________________
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ G  ⋅  ⋅ 
 ⋅  S  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
---------------------------------


## Uninformed search algorithms

### Breath - first search

In [47]:
class FIFO:
    """A simple queue implementation using `collections.deque`."""
    def __init__(self) -> None:
        self.elements: collections.deque[T] = collections.deque()       

    def empty(self) -> bool:
        return not self.elements            #returns true if the queue is empty
    
    def put(self, x: T) -> None:
        self.elements.append(x)

    def get(self) -> T:
        return self.elements.popleft()

def breadth_first_search(graph: Graph, start: Location, goal: Location):
    frontier = FIFO()
    frontier.put(start)

    reached: dict[Location, Optional[Location]] = {}    #reached contains a node and its parent path-wise
    reached[start] = None

    while not frontier.empty():
        current: Location = frontier.get()
        print(f"Visiting {current}")
        if current == goal: #early exit: note how the goal state is recognized as such when it is expanded, not when it is added to the frontier
            break
        for next_location in graph.neighbors(current):
            if next_location not in reached:
                frontier.put(next_location)
                reached[next_location] = current

    return reached

In [48]:
#test bfs
parents = breadth_first_search(g, start, goal)
draw_grid(g, point_to=parents, start=start, goal=goal)
draw_grid(g, path=reconstruct_path(parents, start=start, goal=goal),start=start, goal=goal)

Visiting (1, 4)
Visiting (2, 4)
Visiting (0, 4)
Visiting (1, 3)
Visiting (1, 5)
Visiting (2, 5)
Visiting (2, 3)
Visiting (3, 4)
Visiting (0, 5)
Visiting (0, 3)
Visiting (1, 2)
Visiting (1, 6)
Visiting (3, 5)
Visiting (2, 6)
Visiting (3, 3)
Visiting (2, 2)
Visiting (4, 4)
Visiting (0, 6)
Visiting (0, 2)
Visiting (1, 1)
Visiting (3, 6)
Visiting (4, 5)
Visiting (3, 2)
Visiting (4, 3)
Visiting (2, 1)
Visiting (5, 4)
Visiting (0, 7)
Visiting (0, 1)
Visiting (1, 0)
Visiting (4, 6)
Visiting (5, 5)
Visiting (4, 2)
Visiting (3, 1)
Visiting (5, 3)
Visiting (2, 0)
Visiting (6, 4)
Visiting (0, 8)
Visiting (0, 0)
Visiting (4, 7)
Visiting (5, 6)
Visiting (6, 5)
Visiting (4, 1)
Visiting (5, 2)
Visiting (3, 0)
Visiting (6, 3)
Visiting (7, 4)
Visiting (0, 9)
Visiting (5, 7)
Visiting (4, 8)
Visiting (6, 6)
Visiting (7, 5)
Visiting (5, 1)
Visiting (4, 0)
Visiting (6, 2)
Visiting (7, 3)
Visiting (8, 4)
Visiting (1, 9)
Visiting (5, 8)
Visiting (6, 7)
Visiting (4, 9)
Visiting (7, 6)
Visiting (8, 5)
Visiting

### Depth - first search

In [49]:
class LIFO:
    """A simple queue implementation using `collections.deque`."""
    def __init__(self) -> None:
        self.elements: collections.deque[T] = collections.deque()

    def empty(self) -> bool:
        return not self.elements
    
    def put(self, x: T) -> None:
        self.elements.append(x)

    def get(self) -> T:
        return self.elements.pop()      #THE ONLY DIFFERENCE WITH BFS IS THE RIGHT POP

def depth_first_search(graph: Graph, start: Location, goal: Location):
    frontier = LIFO()
    frontier.put(start)

    reached: dict[Location, Optional[Location]] = {}    #reached contains a node and its parent path-wise
    reached[start] = None

    while not frontier.empty():
        current: Location = frontier.get()
        print(f"Visiting {current}")
        if current == goal: #early exit: note how the goal state is recognized as such when it is expanded, not when it is added to the frontier
            break
        for next_location in graph.neighbors(current):
            if next_location not in reached:
                frontier.put(next_location)
                reached[next_location] = current

    return reached

In [50]:
#test dfs
parents = depth_first_search(g, start, goal)
draw_grid(g, point_to=parents, start=start, goal=goal)
draw_grid(g, path=reconstruct_path(parents, start=start, goal=goal),start=start, goal=goal)

Visiting (1, 4)
Visiting (1, 5)
Visiting (2, 5)
Visiting (2, 6)
Visiting (3, 6)
Visiting (4, 6)
Visiting (5, 6)
Visiting (5, 7)
Visiting (6, 7)
Visiting (6, 8)
Visiting (7, 8)
Visiting (7, 9)
Visiting (8, 9)
Visiting (9, 9)
Visiting (10, 9)
Visiting (10, 8)
Visiting (10, 7)
Visiting (10, 6)
Visiting (9, 6)
Visiting (9, 5)
Visiting (8, 5)
Visiting (8, 4)
Visiting (7, 4)
Visiting (7, 3)
Visiting (6, 3)
Visiting (6, 2)
Visiting (5, 2)
Visiting (5, 1)
Visiting (4, 1)
Visiting (4, 0)
Visiting (3, 0)
Visiting (2, 0)
Visiting (1, 0)
Visiting (1, 1)
Visiting (0, 1)
Visiting (0, 2)
Visiting (0, 3)
Visiting (1, 2)
Visiting (2, 2)
Visiting (3, 2)
Visiting (3, 3)
Visiting (4, 3)
Visiting (4, 4)
Visiting (5, 4)
Visiting (3, 4)
Visiting (2, 3)
Visiting (0, 0)
Visiting (2, 1)
Visiting (3, 1)
Visiting (5, 0)
Visiting (6, 0)
Visiting (7, 0)
Visiting (7, 1)
Visiting (8, 1)
Visiting (8, 2)
Visiting (9, 2)
Visiting (9, 3)
Visiting (10, 3)
Visiting (10, 4)
Visiting (10, 2)
Visiting (10, 1)
Visiting (10, 0)

### Uniform - cost search

In [51]:
import heapq

class PriorityQueue:
    """A simple queue implementation using `collections.deque`."""
    def __init__(self) -> None:
        self.elements: list[tuple[float, T]] = []       #each of the elements contains its cost and the node

    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: T, priority: float) -> None:
        heapq.heappush(self.elements, (priority, item))     #pushes into the heap (thus automatically reordering it!) the new node

    def get(self) -> T:
        return heapq.heappop(self.elements)[1]  #it only returns the node

def uc_search(graph: WeightedGraph, start: Location, goal: Location):
    frontier = PriorityQueue()
    frontier.put(start, 0)

    reached: dict[Location, Optional[Location]] = {}    #this contains node, parent of the node on the path from the source
    cost_so_far: dict[Location, float] = {}             #this expresses the best current g(n) (that is, the cost to reach that node), of all reached nodes
    reached[start] = None
    cost_so_far[start] = 0                              


    while not frontier.empty():
        current: Location = frontier.get()
        #print(f"Visiting {current}")
        if current == goal: #early exit as usual, only when target goal is expanded
            break
        for next_location in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next_location)
            if next_location not in reached or cost_so_far[next_location] > new_cost:
                frontier.put(next_location, new_cost)
                reached[next_location] = current
                cost_so_far[next_location] = new_cost

    return reached

In [52]:
#test ucs
parents = uc_search(g, start, goal)
path = reconstruct_path(parents, start=start, goal=goal)
#draw_grid(g, point_to=parents, start=start, goal=goal)
draw_grid(g, path=path,start=start, goal=goal)
#computing the cost
total = 0
for i in range(len(path) - 1):
    total += g.cost(path[i], path[i+1])
print(total)

_________________________________
 ⋅  @  @  @  @  @  @  ⋅  ⋅  ⋅  ⋅ 
 ⋅  @  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅ @  @  ⋅  ⋅  ⋅ 
 ⋅  @  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅ @  @  ⋅  ⋅ 
 ⋅  @  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ G  ⋅  ⋅ 
 ⋅  S  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
---------------------------------
14


## Informed search algorithms

For informed search algorithms, it is necessary to define an heuristic function $h(n)$.  
I will employ *Manhattan distance* as my heuristic.

### Greedy best - first search

In [53]:
import heapq

class PriorityQueue:
    """A simple queue implementation using `collections.deque`."""
    def __init__(self) -> None:
        self.elements: list[tuple[int, T]] = []       #each of the elements contains its heuristic and the node

    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: T, priority: int) -> None:
        heapq.heappush(self.elements, (priority, item))     #pushes into the heap (thus automatically reordering it!) the new node

    def get(self) -> T:
        return heapq.heappop(self.elements)[1]  #it only returns the node
    
def mdistance(node1: Location, node2: Location) -> int:
    x_dis = abs(node1[0] - node2[0])
    y_dis = abs(node1[1] - node2[1])

    return x_dis + y_dis


def greedy_bfs(graph: WeightedGraph, start: Location, goal: Location):
    frontier = PriorityQueue()
    frontier.put(start, mdistance(start, goal))

    reached: dict[Location, Optional[Location]] = {}    #this contains node, parent of the node on the path from the source
    reached[start] = None        

    while not frontier.empty():
        current: Location = frontier.get()
        #print(f"Visiting {current}")
        if current == goal: #early exit as usual, only when target goal is expanded
            break
        for next_location in graph.neighbors(current):
            if next_location not in reached:
                frontier.put(next_location, mdistance(next_location, goal))
                reached[next_location] = current

    return reached

def path_cost(path: list[Location]) -> int:
    total = 0
    for i in range(len(path) - 1):
        print(f"Weight of path {path[i]} to {path[i+1]}: {g.cost(path[i], path[i+1])}")
        total += g.cost(path[i], path[i+1])
    return total

In [54]:
#test gbfs
parents = greedy_bfs(g, start, goal)
path=reconstruct_path(parents, start=start, goal=goal)
draw_grid(g, point_to=parents, start=start, goal=goal)
draw_grid(g, path=path, start=start, goal=goal)
#computing the cost
print(path_cost(path))

_________________________________
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ∨  ∨  ∨  ∨  ∨  ∨  ∨  ⋅  ⋅  ⋅ 
 >  ∨  <  <  <  <  <  <  G  ⋅  ⋅ 
 >  S  <  ∧  ∧  ∧  ∧  ∧  ⋅  ⋅  ⋅ 
 ⋅  ∧  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
---------------------------------
_________________________________
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅  @  @  @  @  @  @  @  G  ⋅  ⋅ 
 ⋅  S  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
---------------------------------
Weight of path (1, 4) to (1, 3): 1
Weight of path (1, 3) to (2, 3): 1
Weight of path (2, 3) to (3, 3): 1
Weight of path (3, 3) to (4, 3): 5
Weight of path (4, 3) to (5, 3): 5
Weight of

### A*

In [59]:
import heapq

class PriorityQueue:
    """A simple queue implementation using `collections.deque`."""
    def __init__(self) -> None:
        self.elements: list[tuple[float, T]] = []       #each of the elements contains its heuristic and the node

    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: T, priority: float) -> None:
        heapq.heappush(self.elements, (priority, item))     #pushes into the heap (thus automatically reordering it!) the new node

    def get(self) -> T:
        return heapq.heappop(self.elements)[1]  #it only returns the node
    
def mdistance(node1: Location, node2: Location) -> int:
    x_dis = abs(node1[0] - node2[0])
    y_dis = abs(node1[1] - node2[1])

    return x_dis + y_dis


def a_star(graph: WeightedGraph, start: Location, goal: Location):
    frontier = PriorityQueue()
    frontier.put(start, mdistance(start, goal))         #frontier ordered by current best distance up to that node + heuristic up to goal

    reached: dict[Location, Optional[Location]] = {}    #this contains node, parent of the node on the path from the source
    cost_so_far: dict[Location, float] = {}             #this expresses the best current g(n) (that is, the cost to reach that node), of all reached nodes
    reached[start] = None        
    cost_so_far[start] = 0                              

    while not frontier.empty():
        current: Location = frontier.get()
        #print(f"Visiting {current}")
        if current == goal: #early exit as usual, only when target goal is expanded
            break
        for next_location in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next_location)
            if next_location not in reached or cost_so_far[next_location] > new_cost:
                frontier.put(next_location, new_cost + mdistance(next_location, goal))
                cost_so_far[next_location] = new_cost
                reached[next_location] = current

    return reached

def path_cost(path: list[Location]) -> int:
    total = 0
    for i in range(len(path) - 1):
        print(f"Weight of path {path[i]} to {path[i+1]}: {g.cost(path[i], path[i+1])}")
        total += g.cost(path[i], path[i+1])
    return total

In [60]:
#test a*
parents = a_star(g, start, goal)
path=reconstruct_path(parents, start=start, goal=goal)
draw_grid(g, point_to=parents, start=start, goal=goal)
draw_grid(g, path=path, start=start, goal=goal)
#computing the cost
print(path_cost(path))

_________________________________
 ∨  ∨  ∨  ∨  <  <  <  <  <  <  ⋅ 
 ∨  ∨  ∨  ∨  <  ∧  ∧  <  <  <  ⋅ 
 ∨  ∨  ∨  ∨  <  <  ∧  ∧  <  <  ⋅ 
 >  ∨  <  <  <  < ⋅⋅⋅ ∧  G  ⋅  ⋅ 
 >  S  <  <  < ⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ∧  ∧  ∧  <  < ⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ∧  ∧  ∧  <  < ⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ∧ #########⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
---------------------------------
_________________________________
 ⋅  ⋅  ⋅  @  @  @  @  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  @ ⋅⋅⋅⋅⋅⋅ @  @  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  @ ⋅⋅⋅⋅⋅⋅⋅⋅⋅ @  @  ⋅  ⋅ 
 ⋅  @  @  @ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ G  ⋅  ⋅ 
 ⋅  S  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅ ⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅ 
 ⋅ #########⋅⋅⋅⋅⋅⋅ ⋅  ⋅  ⋅  ⋅  ⋅ 
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 
---------------------------------
Weight of path (1, 4) to (1, 3): 1
Weight of path (1, 3) to (2, 3): 1
Weight of path (2, 3) to (3, 3): 1
Weight of path (3, 3) to (3, 2): 1
Weight of path (3, 2) to (3, 1): 1
Weight of